<a href="https://colab.research.google.com/github/SurPaul/stable-diffusion-webui-colab/blob/main/Stable_Diffusion_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Настройка

#@markdown #### 1. Сохранить папки `outputs`, `textual_inversion` и `hypernetworks` на Google Диск? Они будут сохранены в `/Colab Notebooks`.
OUTPUT_DIR_SYNC = False #@param {type: "boolean"}

#@markdown #### 2. [Получите](https://huggingface.co/settings/tokens) токен на Hugging Face 🤗
HUGGINGFACE_TOKEN = "" #@param {type: "string"}

#@markdown #### 3. Выберите версию Stable Diffusion для скачивания:
SD_VERSION = "v1-5-pruned-emaonly.ckpt" #@param ["v1-5-pruned-emaonly.ckpt", "arcane-diffusion-v3.ckpt", "analog-diffusion-1.0.ckpt", "Anything-V3.0-pruned-fp16.ckpt", "Deliberate.safetensors", "dreamlike-diffusion-1.0.ckpt", "dreamlike-photoreal-1.0.ckpt", "dreamlike-photoreal-2.0.ckpt", "mdjrny-v4.ckpt", "redshift-diffusion-v1.ckpt", "waifu-diffusion-v1-3-float16.ckpt"]

#@markdown #### 4. Если вы хотите использовать свою модель на Google Диске, укажите путь до неё, включая имя файла, и версию Stable Diffusion, на которой она основана:
CUSTOM_MODEL = "" #@param {type: "string"}

CUSTOM_MODEL_VERSION = "1.5" #@param ["1.5", "2.0/2.1 512px", "2.0/2.1 768px"]

#@markdown После выбора нужных параметров вы можете нажать `Ctrl + F9` (`Среда выполнения` > `Выполнить всё`), чтобы запустить WebUI.</br>Ссылка на веб-интерфейс (`*.gradio.live`) через несколько минут появится в ячейке `Запуск WebUI`.

sd_models_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"
user_header = f"\"Authorization: Bearer {HUGGINGFACE_TOKEN}\""

!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

from os.path import exists
from google.colab import drive, output
from sys import exit

if OUTPUT_DIR_SYNC or CUSTOM_MODEL:
  if not exists("/content/drive"):
    drive.mount("/content/drive")

  if CUSTOM_MODEL:
    CUSTOM_MODEL = "/content/drive/MyDrive/" + CUSTOM_MODEL
    if not exists(CUSTOM_MODEL):
      exit(f"Модель {CUSTOM_MODEL} не найдена. Проверьте путь до модели на Google Диске или очистите поле CUSTOM_MODEL, чтобы использовать модель, выбранную в SD_VERSION.")

In [ ]:
#@title Скачивание и установка
#@markdown * Подключение Google Диска *(опционально)*;
#@markdown * Установка Torch, Xformers и других зависимостей;
#@markdown * Скачивание Stable Diffusion с Hugging Face 🤗
#@markdown * Веб-интерфейс [AUTOMATIC1111/stable-diffusion-webui](https://github.com/AUTOMATIC1111/stable-diffusion-webui);
#@markdown * Расширение [yfszzx/stable-diffusion-webui-images-browser](https://github.com/yfszzx/stable-diffusion-webui-images-browser) для просмотра результатов из веб-интерфейса;
#@markdown * Расширение [yownas/seed_travel](https://github.com/yownas/seed_travel).

#@title Symlink между GDrive и Outputs
from os import symlink, getenv, environ

!git clone -q https://github.com/AUTOMATIC1111/stable-diffusion-webui
!git clone -q https://github.com/yfszzx/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone -q https://github.com/yownas/seed_travel /content/stable-diffusion-webui/extensions/seed_travel
!git clone -q https://github.com/Extraltodeus/depthmap2mask /content/stable-diffusion-webui/extensions/depthmap2mask

!pip install -q torch torchvision torchaudio xformers moviepy==1.0.3
!pip install --pre -q triton

if OUTPUT_DIR_SYNC:

  src_op = "/content/drive/MyDrive/Colab Notebooks/outputs"
  dst_op = "/content/stable-diffusion-webui/outputs"
  src_ti = "/content/drive/MyDrive/Colab Notebooks/textual_inversion"
  dst_ti = "/content/stable-diffusion-webui/textual_inversion"
  src_hn = "/content/drive/MyDrive/Colab Notebooks/hypernetworks"
  dst_hn = "/content/stable-diffusion-webui/models/hypernetworks"

  if not exists(src_op):
    %mkdir -p "{src_op}"

  if not exists(dst_op):
    symlink(src_op, dst_op)

  if not exists(src_ti):
    %mkdir -p "{src_ti}"

  if not exists(dst_ti):
    symlink(src_ti, dst_ti)

  if not exists(src_hn):
    %mkdir -p "{src_hn}"

  if not exists(dst_hn):
    symlink(src_hn, dst_hn)

output.clear()
conf_dir = "/content/stable-diffusion-webui/config"
conf = f"--config {conf_dir}/v1-inference.yaml"

if CUSTOM_MODEL:
  ckpt = f"--ckpt '{CUSTOM_MODEL}'"
  if CUSTOM_MODEL_VERSION == "2.0/2.1 512px":
    conf = f"--config {conf_dir}/v2-inference.yaml"
  elif CUSTOM_MODEL_VERSION == "2.0/2.1 768px":
    conf = f"--config {conf_dir}/v2-inference-v.yaml"
else:
  ckpt = f"--ckpt {sd_models_dir}/{SD_VERSION}"

if not exists(conf_dir):
  %mkdir {conf_dir}
  !wget --header={user_header} -q https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-inference.yaml -O {conf_dir}/v1-inference.yaml
  !wget -q https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml -O {conf_dir}/v2-inference.yaml
  !wget -q https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O {conf_dir}/v2-inference-v.yaml

if not CUSTOM_MODEL:
  if SD_VERSION == "v1-5-pruned-emaonly.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -O {sd_models_dir}/v1-5-pruned-emaonly.ckpt
      
  elif SD_VERSION == "v1-5-pruned.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt -O {sd_models_dir}/v1-5-pruned.ckpt
      
  elif SD_VERSION == "v2-1_512-ema-pruned.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt -O {sd_models_dir}/v2-1_512-ema-pruned.ckpt
    conf = f"--config {conf_dir}/v2-inference.yaml"
    
  elif SD_VERSION == "v2-1_768-ema-pruned.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -O {sd_models_dir}/v2-1_768-ema-pruned.ckpt
    conf = f"--config {conf_dir}/v2-inference-v.yaml"
    
  elif SD_VERSION == "arcane-diffusion-v3.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/nitrosocke/Arcane-Diffusion/resolve/main/arcane-diffusion-v3.ckpt -O {sd_models_dir}/arcane-diffusion-v3.ckpt
      
  elif SD_VERSION == "analog-diffusion-1.0.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/wavymulder/Analog-Diffusion/resolve/main/analog-diffusion-1.0.ckpt -O {sd_models_dir}/analog-diffusion-1.0.ckpt
      
  elif SD_VERSION == "Anything-V3.0-pruned-fp16.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/Anything-V3.0-pruned-fp16.ckpt -O {sd_models_dir}/Anything-V3.0-pruned-fp16.ckpt
      
  elif SD_VERSION == "Deliberate.safetensors":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/XpucT/Deliberate/resolve/main/Deliberate.safetensors -O {sd_models_dir}/Deliberate.safetensors
      
  elif SD_VERSION == "dreamlike-diffusion-1.0.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/dreamlike-art/dreamlike-diffusion-1.0/resolve/main/dreamlike-diffusion-1.0.ckpt -O {sd_models_dir}/dreamlike-diffusion-1.0.ckpt
      
  elif SD_VERSION == "dreamlike-photoreal-1.0.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/dreamlike-art/dreamlike-photoreal-1.0/resolve/main/dreamlike-photoreal-1.0.ckpt -O {sd_models_dir}/dreamlike-photoreal-1.0.ckpt
      
  elif SD_VERSION == "dreamlike-photoreal-2.0.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/dreamlike-art/dreamlike-photoreal-2.0/resolve/main/dreamlike-photoreal-2.0.ckpt -O {sd_models_dir}/dreamlike-photoreal-2.0.ckpt
      
  elif SD_VERSION == "mdjrny-v4.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt -O {sd_models_dir}/mdjrny-v4.ckpt
      
  elif SD_VERSION == "redshift-diffusion-v1.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/nitrosocke/redshift-diffusion/resolve/main/redshift-diffusion-v1.ckpt -O {sd_models_dir}/redshift-diffusion-v1.ckpt
      
  elif SD_VERSION == "waifu-diffusion-v1-3-float16.ckpt":
    !wget --header={user_header} -cq --show-progress https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float16.ckpt -O {sd_models_dir}/waifu-diffusion-v1-3-float16.ckpt

In [ ]:
#@title Запуск WebUI
%cd /content/stable-diffusion-webui
!python launch.py --share --enable-insecure-extension-access --xformers	{ckpt} {conf}